# calendar.ipynb

## run ./restart-win.sh in git bash 1st


In [ ]:
cd ~/OneDrive/chorus/abv/tech/abv/server
./win-restart.sh

## setup 

In [1]:
## setup
import abv
from importlib import reload
reload(abv)
from pathlib import Path
import os
import json
from bs4 import BeautifulSoup

# run from this relative folder
todo_relative = "../../"
resource_relative = ""
calendar_relative = "./"
local_host = "http://localhost:8000/abv/calendar/"
abv_host = "https://abvchorus.org/"
remote_dir = "public_html/"
cal_name = "calendar.html"
cal_path = f"{calendar_relative}{cal_name}"
link_ref = ""

## workflow 
To deal with calendar update from section leaders
2. take downloaded file from abv leader and save it in ../todo
3. clean (and convert if docx) and place it in resources/ then upload it to abv
4. keep the result["link_href"] resources/... as link_ref
5. enter dateCode, turn calendar to soup, from dateCode make yearmo
6. find month from calendar soup, make ev_mo_soup
7. using dateCode, create ev_day_soup. If ev_day_soup make ev_day_json
8. 
Event for that day already exists
1. if ev_day_json, copy ev_day_json to ev_mod_json
2. EDIT ev_mod_json then turn it into ev_mod_soup
3. replace ev_day_soup with event_mod_soup

Event doesn't exist, new event for that day
1. EDIT new_ev_json
2. create new_ev_html from new_ev_json
3. turn it into new_ev_soup
4. insert_new_ev_in_mo() into ev_mo_soup
Review and complete the update
1. check your work by viewing on localhost or tryit
2. delete any extraneous events and check you work again
3. create a calendar backup
4. upload modified calendar to abv

### download calendar from abv

In [44]:
abv.download_file(local_dir=calendar_relative, remote_dir=remote_dir, filename=cal_name)

url = f"access: {local_host}{cal_name}"
print(url)

access: http://localhost:8000/abv/calendar/calendar.html


### clean(,convert),copy to resources, upload

In [25]:

todo_win_relative = r"todo\ABV REHEARSAL 9 NOV 2025.docx" #from copy relative path
link_path="resources/other/rehearsal_notes/25-26/"

# constructed paths
source_path = os.path.normpath(todo_relative + todo_win_relative)
target_dir = f"{resource_relative}{link_path}"

message, result = abv.upload_cleaned_file(source_path, target_dir, link_path)
print(message)
print(result["link_href"])
link_ref = result["link_href"]

cleaned_path: resources\other\rehearsal_notes\25-26\ABV_REHEARSAL_9_NOV_2025.docx
Copied: ..\..\todo\ABV REHEARSAL 9 NOV 2025.docx -> resources\other\rehearsal_notes\25-26\ABV_REHEARSAL_9_NOV_2025.docx
Converting resources\other\rehearsal_notes\25-26\ABV_REHEARSAL_9_NOV_2025.docx to PDF...


  0%|          | 0/1 [00:00<?, ?it/s]

Successfully converted to: resources\other\rehearsal_notes\25-26\ABV_REHEARSAL_9_NOV_2025.pdf
Removed copied DOC/DOCX file: resources\other\rehearsal_notes\25-26\ABV_REHEARSAL_9_NOV_2025.docx
Uploading resources\other\rehearsal_notes\25-26\ABV_REHEARSAL_9_NOV_2025.pdf to public_html/resources/other/rehearsal_notes/25-26/ABV_REHEARSAL_9_NOV_2025.pdf...
✓ public_html/resources/other/rehearsal_notes/25-26/ABV_REHEARSAL_9_NOV_2025.pdf uploaded from resources\other\rehearsal_notes\25-26\ABV_REHEARSAL_9_NOV_2025.pdf

resources/other/rehearsal_notes/25-26/ABV_REHEARSAL_9_NOV_2025.pdf


### setup - dateCode, yrmo

In [2]:
dateCode = "2026-04-12"
yrmo = dateCode[:7]

### turn calendar to soup

In [3]:
with open(cal_path, 'r', encoding='utf-8') as f:
    cal_html_str = f.read()
    
soup = BeautifulSoup(cal_html_str, 'html.parser')

### find html for month of insertion

In [4]:
ev_mo_soup = soup.find('tr', attrs={"data-yrmo": yrmo})
print(ev_mo_soup.prettify())

<tr data-yrmo="2026-04">
 <td class="month-column">
  <span class="month-name">
   April
  </span>
  <span class="month-year">
   2026
  </span>
 </td>
 <td class="weekly-column">
  <!-- Add events for April 2026 here -->
  <div class="event-item" data-date="2026-04-12">
   <span class="event-date">
    4/12
   </span>
   <span class="event-title">
    Rehearsal
   </span>
   <span class="event-description">
   </span>
  </div>
  <div class="event-item" data-date="2026-04-19">
   <span class="event-date">
    4/19
   </span>
   <span class="event-title">
    Rehearsal
   </span>
   <span class="event-description">
   </span>
  </div>
 </td>
</tr>



### find ev_day_soup in ev_mo_soup

In [6]:
ev_day_soup = ev_mo_soup.find('div', attrs={"data-date": dateCode})
if ev_day_soup:
    print(ev_day_soup.prettify())
    ev_day_json = abv.event_html_to_json(ev_day_soup)
    print(json.dumps(ev_day_json, indent=2))

<div class="event-item" data-date="2026-04-12">
 <span class="event-date">
  4/12
 </span>
 <span class="event-title">
  Rehearsal
 </span>
 <span class="event-description">
 </span>
</div>

{
  "dateCode": "2026-04-12",
  "dateStr": "4/12",
  "description": "",
  "title": "Rehearsal",
  "links": []
}


### EDIT ev_mod_json (if ev_day_json exists) EDIT THIS

### reheasal zoom and other external links

In [7]:
external = False
if external:
    ext_lnk= "https://us02web.zoom.us/rec/share/Pob5SUgbBSMeBu-EU0k_pyKO4cVLpruF3-nmHN6YzH3K6-xIumF1GNoxNj7YLeyh.4hO2MbepCBUS8HEA"
    ext_descr = " Passcode: D8fx+D0L "

### EDIT additional manual EDIT

In [9]:
if ev_day_json:
    ev_mod_json = ev_day_json
    if external:
        ev_mod_json["links"].append({
            "href": ext_lnk,
            "lnkTxt": " Recording ",
            "lnkDescr": ext_descr
        })
    if len(link_ref) >0:
        ev_mod_json["links"].append({
            "href": link_ref,
            "lnkTxt": " NOTES ",
            "lnkDescr": ""
        })
    # ev_mod_json["description"] = "" 
    # ev_mod_json["links"] = []
# 
    # ev_mod_json["links"].append({
    #     "href": "",
    #     "lnkTxt": "Recording",
    #     "lnkDescr": "Passcode:"
    # })
    
    # ev_mod_json["links"].append({
    #     "href": link_ref,
    #     "lnkTxt": " NOTES",
    #     "lnkDescr": ""
    # })


    # ev_mod_json["links"].append({
    #     "href": link_ref,
    #     "lnkTxt": "CONCERT POSTER",
    #     "lnkDescr": "Koleinu concert PDF"
    # })

    # ev_mod_json["links"][0]["lnkDescr"] = "Ryeah right"

    # ev_mod_json["links"]=[]
    # ev_mod_json["links"] = [
    #     {
    #         "href": link_ref,
    #         "lnkTxt": "CONCERT POSTER ",
    #         "lnkDescr": " Koleinu concert PDF"
    #     }
    # ]
    
name = "ev_mod_json"
print(f"{name} = ")
ev_mod_json = json.dumps(ev_mod_json, indent=2)
print(ev_mod_json)
abv.copy_output_to_next_cell(ev_mod_json, name)


ev_mod_json = 
{
  "dateCode": "2026-04-12",
  "dateStr": "4/12",
  "description": "",
  "title": "Rehearsal",
  "links": []
}


In [10]:
ev_mod_json = {
  "dateCode": "2026-04-12",
  "dateStr": "4/12",
  "description": "",
  "title": "Rehearsal not",
  "links": []
} 

In [11]:
print(json.dumps(ev_mod_json, indent=2) )

{
  "dateCode": "2026-04-12",
  "dateStr": "4/12",
  "description": "",
  "title": "Rehearsal not",
  "links": []
}


### ev_mod_json -> ev_mod_html -> ev_mod_soup

### ev_day_soup.replace_with(ev_mod_soup)

In [12]:
if ev_day_json:   
    print(json.dumps(ev_mod_json, indent=2))
    ev_mod_html = abv.create_event_html_from_json(ev_mod_json)
    ev_mod_soup = BeautifulSoup(ev_mod_html, 'html.parser')
    print(ev_mod_soup.prettify())
    ev_day_soup.replace_with(ev_mod_soup)
    print(ev_mo_soup.prettify())

{
  "dateCode": "2026-04-12",
  "dateStr": "4/12",
  "description": "",
  "title": "Rehearsal not",
  "links": []
}
<div class="event-item" data-date="2026-04-12">
 <span class="event-date">
  4/12
 </span>
 <span class="event-title">
  Rehearsal not
 </span>
 <span class="event-description">
 </span>
</div>

<tr data-yrmo="2026-04">
 <td class="month-column">
  <span class="month-name">
   April
  </span>
  <span class="month-year">
   2026
  </span>
 </td>
 <td class="weekly-column">
  <!-- Add events for April 2026 here -->
  <div class="event-item" data-date="2026-04-12">
   <span class="event-date">
    4/12
   </span>
   <span class="event-title">
    Rehearsal not
   </span>
   <span class="event-description">
   </span>
  </div>
  <div class="event-item" data-date="2026-04-19">
   <span class="event-date">
    4/19
   </span>
   <span class="event-title">
    Rehearsal
   </span>
   <span class="event-description">
   </span>
  </div>
 </td>
</tr>



# new event-item

#### EDIT create a new_ev_json EDIT

In [42]:
if not ev_day_json:
    # create new event EDIT THIS event
    links = []
    if link_ref:
        links.append({
            "href": link_ref,
            "lnkTxt": "Koleinu Cocert Poster",
            "lnkDescr": ""
        })
    new_ev_json = {
        "dateCode": adatcode,
        "dateStr": f"{adatcode[5:7]}/{adatcode[8:10]}",
        "title": "Rehearsal",
        "description": "",
        "links": links
    }
    print(json.dumps(new_ev_json, indent=2))
else:
    print("No new event created, existing event is being modified")

No new event created, existing event is being modified


### new_ev_json -> new_ev_html -> new_ev_soup 

### insert_new_ev_in_mo(ev_mo_soup, new_ev_soup) 

In [ ]:
if not ev_day_json:
    new_ev_html = abv.create_event_html_from_json(new_ev_json)
    new_ev_soup = BeautifulSoup(new_ev_html, 'html.parser')
    abv.insert_new_ev_in_mo(ev_mo_soup, new_ev_soup) 

# Check and Save

### save your work - soup -> calendar

In [13]:
soup.prettify()
with open(cal_path, 'w', encoding='utf-8') as f:
    f.write(str(soup))
    
url = f"access: {local_host}{cal_name}"
print(url)

access: http://localhost:8000/abv/calendar/calendar.html


### review your work - click links

### delete events if needed

In [44]:
date_codes = ["2025-12-13"]
soup, html = abv.delete_events(cal_path, date_codes)

with open(cal_path, 'w', encoding='utf-8') as f:
    f.write(str(soup))
    
url = f"access: {local_host}{cal_name}"
print(url)

Event not found: 2025-12-13
Deleted 0 out of 1 events
access: http://localhost:8000/abv/calendar/calendar.html


### create a backup on abv

In [17]:
cal_name = "calendar.html"
remote_dir = "public_html/"
original_path = f"{remote_dir}{cal_name}"
backup_path = f"{remote_dir}calendar_backup.html"
abv.create_remote_backup(original_path, backup_path)

'✅ Copied public_html/calendar.html to public_html/calendar_backup.html\n'

### upload calendar to abv

In [18]:
abv.upload_afile(cal_path, original_path)

url= f"access: {abv_host}{cal_name}"
print(url) 
url = f"access: {local_host}{cal_name}"
print(url)

access: https://abvchorus.org/calendar.html
access: http://localhost:8000/abv/calendar/calendar.html


In [38]:
# Cell A
from IPython import get_ipython
import pprint

obj = {"a": 1, "b": [1, 2, 3]}
p = pprint.pformat(obj)
# insert a new code cell below with the object's literal assigned to 'obj_copy'
get_ipython().set_next_input("obj = " + p, replace=False)
obj  # still displays the original

{'a': 1, 'b': [1, 2, 3]}

In [ ]:
obj = {'a': 1, 'b': [1, 2, 3]}

In [ ]:
obj_copy = {'a': 1, 'b': [1, 2, 3]}

In [46]:
def copy_output_to_next_cell(var):
    from IPython import get_ipython
    import pprint
    p = pprint.pformat(var)
    get_ipython().set_next_input(f"var = {var} " , replace=False)

In [50]:
obj = {"a": 1, "b": [1, 2, 3]}
obj = json.dumps(obj, indent=2)
print(obj)
copy_output_to_next_cell(obj)


{
  "a": 1,
  "b": [
    1,
    2,
    3
  ]
}


In [ ]:
var = {
  "a": 1,
  "b": [
    1,
    2,
    3
  ]
} 

In [ ]:
var = {
  "a": 1,
  "b": [
    1,
    2,
    3
  ]
} = 

In [ ]:
var = '{\n  "a": 1,\n  "b": [\n    1,\n    2,\n    3\n  ]\n}'

if you have a function

```python   
def copy_output_to_next_cell(var):
    from IPython import get_ipython
    import pprint
    p = pprint.pformat(var)
    get_ipython().set_next_input(f"var = {var} " , replace=False)
```
and you run it like this
```python
obj = {"a": 1, "b": [1, 2, 3]}
obj = json.dumps(obj, indent=2)
print(obj)
copy_output_to_next_cell(obj)
```

how do you get it to put in the next cell

```python
obj = {
  "a": 1,
  "b": [
    1,
    2,
    3
  ]
} 
```
instead of

```python
var = {
  "a": 1,
  "b": [
    1,
    2,
    3
  ]
} 
```